# Índice de contenidos
1. Antes de empezar:

2. Reto 1 - Importar y describir el conjunto de datos

    2.0.0.1 Explore el conjunto de datos con técnicas matemáticas y de visualización. ¿Qué encuentra?

3. Reto 2 - Limpieza y transformación de datos

4. Reto 3 - Preprocesamiento de datos

    4.0.0.1 Utilizaremos el StandardScaler de sklearn.preprocessing y escalaremos nuestros datos. Lea más sobre StandardScaler aquí.

5. Reto 4 - Agrupación de datos con K-Means

6. Reto 5 - Agrupación de datos con DBSCAN

7. Reto 6 - Comparar K-Means con DBSCAN

8. Reto adicional 2 - Cambiar el número de clusters de K-Means

9. Bonus Challenge 3 - Cambiar DBSCAN eps y min_samples

# Antes de empezar:
- Lee el archivo README.md
- Comenta todo lo que puedas y utiliza los recursos del archivo README.md
- ¡Feliz aprendizaje!

In [ ]:
# Import your libraries:

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings                                              
from sklearn.exceptions import DataConversionWarning          
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Desafío 1 - Importar y describir el conjunto de datos

En este laboratorio, utilizaremos un conjunto de datos que contiene información sobre las preferencias de los clientes. Analizaremos cuánto gasta cada cliente en un año en cada subcategoría de la tienda de comestibles e intentaremos encontrar similitudes mediante la agrupación.

El origen del conjunto de datos es [aquí](https://archive.ics.uci.edu/ml/datasets/wholesale+customers).

In [ ]:
# loading the data: Wholesale customers data
whole = pd.read_csv('../data/Wholesale customers data.csv')

#### Explora el conjunto de datos con técnicas matemáticas y de visualización. ¿Qué encuentras?

Lista de comprobación:

* ¿Qué significa cada columna?
* ¿Hay datos categóricos que convertir?
* ¿Hay que eliminar datos que faltan?
* Colinealidad de columnas: ¿hay correlaciones altas?
* Estadísticas descriptivas: ¿hay que eliminar algún valor atípico?
* Distribución de los datos por columnas: ¿está sesgada la distribución?
* Etc.

Información adicional: Hace más de un siglo, un economista italiano llamado Vilfredo Pareto descubrió que aproximadamente el 20% de los clientes representan el 80% de las ventas minoristas típicas. Esto se denomina [principio de Pareto](https://en.wikipedia.org/wiki/Pareto_principle). Compruebe si este conjunto de datos presenta esta característica.

In [ ]:
whole.info()
whole.Region.value_counts().sort_index()
whole.describe()

In [ ]:
numericals = whole.select_dtypes(np.number)
corr = numericals.corr()
display(corr)
# Create a heatmap to visualize the correlation matrix
sns.set_context("poster") # Set the Seaborn context to "poster" for larger text and figures
sns.set(rc={"figure.figsize": (12., 6.)}) # Set the default figure size for Seaborn plots
sns.set_style("whitegrid") # Set the Seaborn style to "whitegrid" for a white background with gridlines
sns.heatmap(corr.round(2), annot=True)

In [ ]:
fig, axs = plt.subplots(1, 6, figsize=(15, 6), sharey=True)
sns.boxplot(y='Fresh', data=whole, ax=axs[0])
axs[0].set_title('Fresh')
axs[0].set_ylabel('m.u.')
sns.boxplot(y='Milk', data=whole, ax=axs[1])
axs[1].set_title('Milk')
sns.boxplot(y='Grocery', data=whole, ax=axs[2])
axs[2].set_title('Grocery')
sns.boxplot(y='Frozen', data=whole, ax=axs[3])
axs[3].set_title('Frozen')
sns.boxplot(y='Detergents_Paper', data=whole, ax=axs[4])
axs[4].set_title('Detergents_Paper')
sns.boxplot(y='Delicassen', data=whole, ax=axs[5])
axs[5].set_title('Delicassen')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) #rect=[left, bottom, right, top]
fig.suptitle('Continuous features')

**OBSERVACIONES:**

+ Channel: columna categórica con dos valores posibles, Horeca (Hotel/Restaurant/Café) o Retail.
>```terminal
>Channel
>1    298    Horeca
>2    142    Retail
>```
+ Region: columna target/etiqueta, de tipo categórico, con Lisbon, Oporto o Other Region como valores posibles.
>```terminal
>Region
>1     77   Lisbon
>2     47   Oporto
>3    316   Other Region
>```
+ Las demás columnas (Fresh, Milk, Grocery, Frozen, Detergents_Paper, Delicassen) son contínuas, de tipo _integer_, y representan el gasto anual en unidades de moneda o _monetary units_ (m.u.).


+ Todos las columnas de valores continuos tienen una gran variabilidad (valor mínimo y máximo muy dispares).

+ Los datos categóricos ya están en formato numérico; no hay que convertirlos.

+ Tampoco faltan datos (no hay valores Null en ninguna columna; todas tienen 440 filas).

+ Parece haber una alta correlación entre Grocery y Detergents_Paper (92%).

+ Todas las características contínuas parecen tener valores atípicos, muy especialmente Fresh, con un valor que supera en más de 4 veces el valor promedio.

+ El gasto en Frozen, Detergents_Paper y Delicassen es notoriamente más bajo que en el resto de productos.


# Reto 2 - Limpieza y transformación de datos

Si tu conclusión del reto anterior es que los datos necesitan limpieza/transformación, hazlo en las celdas de abajo. Sin embargo, si su conclusión es que los datos no necesitan ser limpiados o transformados, no dudes en saltarte este reto. Si optas por esta última opción, explica los motivos.

In [ ]:
whole[whole['Fresh']>4*whole['Fresh'].mean()].sort_values(by='Fresh', ascending=False)
# whole[whole['Grocery']>4*whole['Grocery'].mean()].sort_values(by='Grocery', ascending=False)
# whole[whole['Milk']>4*whole['Milk'].mean()].sort_values(by='Milk', ascending=False)
# whole[whole['Frozen']>4*whole['Frozen'].mean()].sort_values(by='Frozen', ascending=False)

**OBSERVACIONES**

+ Aunque algunos valores de Fresh superan en más de 4 veces el promedio, puesto que esos datos están asociados a Other Regions, es posible que sean acumulados de varias ciudades/regiones y por tanto el dato sea válido. Lo mismo para el valor más atípico de Grocery.

# Reto 3 - Preprocesamiento de datos

Uno de los problemas del conjunto de datos es que los rangos de valores son notablemente diferentes en las distintas categorías (por ejemplo, `Fresh` y `Grocery` en comparación con `Detergents_Paper` y `Delicassen`). Si hiciste esta observación en el primer reto, ¡has hecho un gran trabajo! Esto significa que no sólo has completado las preguntas de bonificación en el anterior laboratorio de Aprendizaje Supervisado, sino que también has investigado en profundidad sobre [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). ¡Sigue trabajando así de bien!

Diversos rangos de valores en diferentes características podrían causar problemas en nuestra agrupación. La forma de reducir el problema es mediante el escalado de características. Volveremos a utilizar esta técnica con este conjunto de datos.

#### Utilizaremos el `StandardScaler` de `sklearn.preprocessing` y escalaremos nuestros datos. Lee más sobre `StandardScaler` [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*Después de escalar tus datos, asigna los datos transformados a una nueva variable `customers_scale`.

In [ ]:
X = whole  # Feature data
#y = None  # Target labels (not used in K-means clustering)

from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(X) 
customers_scale = transformer.transform(X) 
customers_scale

# Reto 4 - Agrupación de datos con K-Means

Ahora vamos a agrupar los datos con K-Means primero. Inicia el modelo K-Means, luego ajusta tus datos escalados. En los datos devueltos por el método `.fit`, hay un atributo llamado `labels_` que es el número de cluster asignado a cada registro de datos. Lo que puede hacer es asignar estas etiquetas de nuevo a `customers` en una nueva columna llamada `customers['labels']`. Entonces verá los resultados de cluster de los datos originales.

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

# Instantiate the KMeans model
# random_state=42 is used for reproducibility of results
km = KMeans(random_state=42)

# Instantiate the KElbowVisualizer with the KMeans model
# k=(2,10) indicates the range of number of clusters to try (from 2 to 10)
visualizer = KElbowVisualizer(km, k=(2,10))

# Fit the visualizer to the data
# This will run K-means clustering for each value of k and calculate the distortion score for each
clustering = visualizer.fit(customers_scale)

# Render the plot
# The Elbow plot displays the distortion score for each k
# The point where the distortion score starts to level off ('elbow') is the recommended number of clusters
visualizer.show()

In [ ]:
km_labels = pd.Series(clustering.labels_)
print('Número de clusters:', km_labels.nunique())
km_labels.value_counts()

# Conclusión: El código de arriba nos devuelve los datos para el máximo valor de k que el algoritmo debe de haber probado.

In [ ]:
# A ver qué nos da Silhouette
from yellowbrick.cluster import SilhouetteVisualizer

# Setting up the matplotlib figure with multiple subplots
fig, ax = plt.subplots(5, 2, figsize=(15,8))

# Loop through different numbers of clusters (from 2 to 5)
for i in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    # Create KMeans instance for different number of clusters
    # 'k-means++' for smart centroid initialization, 10 different centroid initializations
    # 100 iterations max for each run, and set a random state for reproducibility
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)

    # Determine the position of the subplot
    q, mod = divmod(i, 2)

    # Create a SilhouetteVisualizer with the KMeans instance
    # Colors are set to 'yellowbrick' palette, and the subplot ax is defined
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])

    # Fit the visualizer to the data to produce the silhouette plot
    visualizer.fit(customers_scale)

# Display the plot
plt.tight_layout()
plt.show()

### Viendo el elbow pododríamos escoger 2 como el número de clusters correctos

In [ ]:
#Hmmm... En realidad el codo estaba en k = 7.... O como mucho en k = 5.... k = 2 ???
kmeans_2 = KMeans(n_clusters=2).fit(customers_scale)

labels = kmeans_2.predict(customers_scale)

clusters = kmeans_2.labels_.tolist()

In [ ]:
whole['labels'] = clusters
whole

Cuenta los valores en `labels`.

In [ ]:
print('Valores en labels (literalmente):', len(labels))
print('Valores únicos de cluster:', whole['labels'].nunique())
whole['labels'].value_counts()

# Reto 5 - Clustering de datos con DBSCAN

Ahora vamos a agrupar los datos utilizando DBSCAN. Utiliza `DBSCAN(eps=0.5)` para iniciar el modelo y, a continuación, ajusta los datos escalados. En los datos devueltos por el método `.fit`, asigna las `labels_` de nuevo a `customers['labels_DBSCAN']`. Ahora tus datos originales tienen dos etiquetas, una de K-Means y la otra de DBSCAN.

In [ ]:
from sklearn.cluster import DBSCAN 

# Applying DBSCAN
# eps: The maximum distance between two samples for them to be considered as in the same neighborhood
# min_samples: The number of samples in a neighborhood for a point to be considered as a core point
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(customers_scale)

whole['labels_DBSCAN'] = clusters
whole

Cuenta los valores en `labels_DBSCAN`.

In [ ]:
print('Valores en labels (literalmente):', len(clusters))
print('Valores únicos de cluster:', whole['labels_DBSCAN'].nunique())
whole['labels_DBSCAN'].value_counts()

# N.B. -1 in clusters represents outliers detected by DBSCAN !!! 
# Esta etiqueta significa que no encajan bien en ningún cluster basándose en los valores de `eps` y `min_samples`.

# Reto 6 - Comparar K-Means con DBSCAN

Ahora queremos comparar visualmente cómo K-Means y DBSCAN han agrupado nuestros datos. Crearemos gráficos de dispersión para varias columnas. Para cada uno de los siguientes pares de columnas, traza un gráfico de dispersión utilizando `labels` y otro utilizando `labels_DBSCAN`. Ponlos uno al lado del otro para compararlos. ¿Qué algoritmo de agrupación tiene más sentido?

Columnas a visualizar:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualice `Detergentes_Papel` como X y `Leche` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
whole.head()

In [ ]:
whole_scaled = pd.DataFrame(customers_scale, columns=whole.columns[:-2])
whole_scaled

In [ ]:
# def plot(x,y,hue):
#     sns.scatterplot(x=x, 
#                     y=y,
#                     hue=hue)
#     plt.title('Detergents Paper vs Milk ')
#     return plt.show();

def plot(x,y,hue,axs,num):
    sns.scatterplot(x=whole_scaled.loc[:, x], 
                    y=whole_scaled.loc[:, y],
                    c=hue, cmap='viridis', marker='o', edgecolor='k', s=50, ax=axs[num])
    axs[num].set_xlabel(x + ' (scaled)')
    axs[num].set_ylabel(y + ' (scaled)')    
    return #plt.show();

In [ ]:
# plt.figure(figsize=(10, 7))
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]

# Método A) Scaled X, como en los apuntes
# plt.scatter(customers_scale[:, 6], customers_scale[:, 3], c=clusters, cmap='viridis', marker='o', edgecolor='k', s=50)

# Método B) X
# plt.scatter(whole.loc[:, 'Detergents_Paper'], whole.loc[:, 'Milk'], c=clusters, cmap='viridis', marker='o', edgecolor='k', s=50)

# Método A mejorado) Scaled X using function
#plot('Detergents_Paper', 'Milk', clusters, 'Detergents_Paper vs Milk')

xcol = 'Detergents_Paper'
ycol = 'Milk'
plot(xcol, ycol, labels, axs, 0)  # K-Means
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

Visualice `Grocery` como X y `Fresh` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]
xcol = 'Grocery'
ycol = 'Fresh'
plot(xcol, ycol, labels, axs, 0)  # K-Means
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

Visualice `Frozen` como X y `Delicassen` como Y mediante `labels` y `labels_DBSCAN` respectivamente

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]
xcol = 'Frozen'
ycol = 'Delicassen'
plot(xcol, ycol, labels, axs, 0)  # K-Means
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

Vamos a utilizar un groupby para ver cómo la media difiere entre los grupos. Agrupamos `customers` por `labels` y `labels_DBSCAN` respectivamente y calculamos las medias de todas las columnas.

In [ ]:
print('Valor promedio de todas las columnas:')
whole.describe().iloc[1]

In [ ]:
whole.groupby(['labels']).mean()

In [ ]:
whole.groupby(['labels_DBSCAN']).mean()

¿Qué algoritmo funciona mejor?

**OBSERVACIONES**

En la prueba que se ha realizado (K-Means con n-clusters=2, según ponía el enunciado y el código preexistente), parece que funciona mejor DBSCAN ya que los valores promedios de cada cluster están más cercanos a los valores promedios de las columnas. 

Podríamos probar K-Means con n-clusters=7, que quizá nos daría una clasificación más parecida a la que nos está dando DBSCAN.

Por lo general, K-Means funciona bien con clusters bien definidos y sin mucho ruido (valores atípicos). Y, precisamente, no es el caso ya que los datos presentan bastantes valores atípicos, muy alejados del promedio.

# Bonus Challenge 2 - Cambiar el número de clusters de K-Means

Como hemos mencionado antes, no tenemos que preocuparnos por el número de clusters con DBSCAN porque lo decide automáticamente en función de los parámetros que le enviemos. Pero con K-Means, tenemos que suministrar el parámetro `n_clusters` (si no se suministra `n_clusters`, el algoritmo utilizará `8` por defecto). Debe saber que el número óptimo de clusters varía en función del conjunto de datos. K-Means puede funcionar mal si se utiliza un número incorrecto de clusters.

En el aprendizaje automático avanzado, los científicos de datos prueban diferentes números de clusters y evalúan los resultados con medidas estadísticas (leer [aquí](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). Hoy no vamos a utilizar medidas estadísticas, sino nuestros ojos. En las celdas de abajo, experimenta con distintos números de conglomerados y visualízalos con gráficos de dispersión. ¿Qué número de clusters parece funcionar mejor para K-Means?

In [ ]:
kmeans_7 = KMeans(n_clusters=7).fit(customers_scale)
labels_7 = kmeans_7.predict(customers_scale)
clusters_7 = kmeans_7.labels_.tolist()

whole['labels_k7'] = clusters_7

print('Valores en labels (literalmente):', len(labels_7))
print('Valores únicos de cluster:', whole['labels_k7'].nunique())
whole['labels_k7'].value_counts()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]

xcol = 'Detergents_Paper'
ycol = 'Milk'
plot(xcol, ycol, labels_7, axs, 0)  # K-Means n-clusters=7
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]
xcol = 'Grocery'
ycol = 'Fresh'
plot(xcol, ycol, labels_7, axs, 0)  # K-Means n-clusters=7
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8), sharey=True)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) #rect=[left, bottom, right, top]
xcol = 'Frozen'
ycol = 'Delicassen'
plot(xcol, ycol, labels_7, axs, 0)  # K-Means n-clusters=7
plot(xcol, ycol, clusters, axs, 1)  # DBSCAN
fig.suptitle(xcol + ' vs ' + ycol)
plt.show()

**OBSERVACIONES:**

* Viendo los gráficos del k-means observo que estableciendo un número mayor de aglomerados consigue agrupar mejor los valores extremos.

# Bonus Challenge 3 - Cambiar `eps` y `min_samples` de DBSCAN

Experimenta cambiando los parámetros `eps` y `min_samples` de DBSCAN. Mira cómo difieren los resultados con la visualización de gráficos de dispersión.

In [ ]:
# Your code here


**Tus observaciones aquí**

    + El DBscan ajustado...
    
